### This notebook plots the model predictions on the features of an inputs file.###

In [ ]:
import numpy as np
import math
import keras
import matplotlib.pyplot as plt
import keras.backend as K
import sys
sys.path.insert(0, '../utils')
from utils import *
sys.path.insert(0,'../preprocessing')
from data_config import config as d_config
sys.path.insert(0, '../train')
from model_configs import get_weighted_bce
from keras.utils.generic_utils import get_custom_objects

In [ ]:
#get file numbers
data_config = d_config()

fs = data_config.fs
num_features = data_config.num_features

print('Train files: ', data_config.train_files)
print('Val files: ', data_config.val_files)
print('Test files: ',data_config.test_files)

In [ ]:
file_num = 5

features = np.load('../preprocessing/numpy_data/inputs/inputs_' + str(file_num) + '.npy')
labels = np.load('../preprocessing/numpy_data/labels/labels_' + str(file_num) + '.npy')

totalSeconds = int(len(labels))/fs

print ('Seconds of lunge data: ', totalSeconds)
print ('Hours: ', totalSeconds/3600)

In [ ]:
#defining custom loss function
pos_weight = 1 #this setting doesn't matter here, just helps to load the model
weighted_bce = get_weighted_bce(pos_weight)
get_custom_objects().update({"weighted_bce": weighted_bce})

In [ ]:
#load label model

#SET THESE: the folder and the name of the desired label model
folder = 'feed_forward_Sat_Aug__3_11-51-22_2019'
model_name = 'ep_2_tp_0.983_fp_0.0_f_1_0.991_f_2_0.986_chain_2_thresh_0.5'

flattened_input = True #true for feed forward, false for resnet

model_name_split = model_name.split('_')
thresholds = [float(model_name_split[-1])]
chaining_dists = [float(model_name_split[-3])]
model = keras.models.load_model('../models/label_models/' + folder + '/' + model_name,
                               custom_objects={'loss': weighted_bce })

In [ ]:
correction_model = None

In [ ]:
#run this cell if using correction model
def avgabs(y_true,y_pred): ##in seconds (if perturbation_max = 5*fs)
    return K.mean(K.abs(5*(y_true - y_pred)))

correction_model = keras.models.load_model('../models/correction_models/correction_model.h5', 
                                           custom_objects={'avgabs': avgabs})

In [ ]:
# Plot the features with lunge labels from startTime to endTime
startTime = 100
endTime = 500
windowTime = endTime-startTime

startSample = startTime*fs
numSamples = windowTime*fs


selected_features = features[startSample:startSample+numSamples,:]
selected_labels = labels[startSample:startSample+numSamples]

t = np.linspace(startTime,startTime+windowTime,numSamples)
tmp = np.where(selected_labels == 1)[0]
markers = [tmp[i] for i in range(len(tmp))]

plt.rcParams['lines.markersize'] = 10
h = 4 #fig width/height
w = 15
colors = ['r', 'b', 'g', 'k', 'm']

for i in range(num_features):
    plt.figure(figsize=(w, h))
    x = selected_features[:, i]
    plt.plot(t,x,marker='o', color=colors[i%len(colors)], label='Feature {}'.format(i), markevery=markers)
    plt.xlabel('Time (seconds)')
    plt.legend()
plt.show()

In [ ]:
print('The lunge times are :')
print (np.where(labels == 1)[0]/fs)

In [ ]:
pred_labels, __ = get_predictions(features, model, flattened_input, correction_model)
pred_labels = np.array(pred_labels)

In [ ]:
# Plot the features with lunge labels and model predictions from startTime to endTime
startTime = 100
endTime = 500
windowTime = endTime-startTime

startSample = startTime*fs
numSamples = windowTime*fs


selected_features = features[startSample:startSample+numSamples,:]
selected_labels = labels[startSample:startSample+numSamples]

t = np.linspace(startTime,startTime+windowTime,numSamples)

tmp = np.where(selected_labels == 1)[0]
true_label_markers = [tmp[i] for i in range(len(tmp))]
prediction_markers = [s - startSample for s in pred_labels if s>= startSample and s<startSample+numSamples]

plt.rcParams['lines.markersize'] = 10
h = 4 #fig width/height
w = 15

colors = ['r', 'b', 'g', 'k', 'm']

for i in range(num_features):
    fig = plt.figure(figsize=(w, h))
    x = selected_features[:,i]
    plt.plot(t,x,marker='o', color=colors[i%5], label='Feature {}'.format(i), markevery=true_label_markers)
    plt.scatter(t[prediction_markers],x[prediction_markers], label='Model Prediction', marker='^', color='m',s = 200)
    plt.xlabel('Time (seconds)')
    plt.legend()
plt.show()